# Analysis of images of perforated foils and calculation of diffusive conductances
Author: stan.schymanski@env.ethz.ch

# Functions to read data files and compute conductances

In [1]:
%%capture storage
# The above redirects all output of the below commands to the variable 'storage' instead of displaying it.
# It can be viewed by typing: 'storage()'

# Setup of worksheet, incl. importing of packages and defining general custom functions
load('temp/Worksheet_setup.sage')
# From stomatal_cond_eqs
load_session('temp/stomatal_cond_eqs.sobj')
import csv

In [2]:
path_data = 'data/perforated_foils/'

In [3]:
# Adding foil thickness to dictionary of general constants, cdict:
cdict[d_p] = 25e-6   # Foil thickness was 25 um.
dict_print(cdict)

M_N2            0.0280000000000000
M_O2            0.0320000000000000
M_w             0.0180000000000000
Pr              0.710000000000000
R_mol           8.31447200000000
a_sh            2              
c_pa            1010           
d_p             0.0000250000000000000
epsilon_l       1              
g               9.81000000000000
lambda_E        2.45000000000000e6
sigm            5.67000000000000e-8


## Functions to read bump data and compute conductances

In [4]:
fname =path_data+'63_7/63_7_1a_bump_a.csv'
fh = open(fname)
for i in srange(10):
    row = fh.readline()
    print row

CSV Version,1

"File name","63_7_1a.rpt"

"Title",""

"Measurement date","8/19/2013"

"Measurement time","4:18:28 PM"

"User mode","Easy"

"Objective lens","Standard lens 10.0","x"

"NA","0.300"

"Size","Standard"

"Mode","Surface profile"



In [5]:
def stat_bump(fname, delimiter1=','):
    '''
    Returns statistics of holes in image, given name of csv file
    '''

    # Determining number of rows to skip
    with open(fname, 'U') as fh:
        data = []
        # Counting rows to be skipped
        skiplines = 0
        dataline = fh.readline()
        #print dataline
        while(len(dataline)<100):
            skiplines = skiplines+1
            dataline = fh.readline()
            if len(dataline) == 0:
                return 'empty line encountered'

    #print skiplines

    # Reading whole file
    with open(fname, 'rb') as fh:
        reader = csv.reader(fh, delimiter=delimiter1, quotechar='"')
        for i in srange(skiplines):
            reader.next()  


        # Creating list of column names
        nameslist = reader.next()
        #print nameslist[0]
        ncols = len(nameslist)
        nameslist1 = []
        name = ""
        for i in srange(ncols):
            if nameslist[i]!= "":
                name = nameslist[i]
            else:
                name = "unit_"+name
            nameslist1.append(name)
        formatslist = []
        for name in nameslist1:
            fmt = 'f8'
            if (name[0:4] == 'unit') or (name[0:4] == 'Comm'):
                fmt = 'S100'
            formatslist.append(fmt)  

        # Saving data in array
        data = []
        for row in reader:
            if len(row) != len(nameslist1):
                break
            data.append(tuple(row))  

    try:    data = np.array(data,dtype = zip(nameslist1,formatslist))
    except:
        print data[0]
        print nameslist1
        print formatslist
    
    # Removing artefacts (areas smaller than 1% of median)
    med_area = np.median(data['Surface area'])
    pos = np.where(data['Surface area'] < med_area/100.)
    data1 = np.delete(data,pos)
    data = data1.copy()

    porosity = sum(data['Area ratio'])
    area_av = np.average(data['C.S. area'])
    density_mm = (porosity/100.)/(area_av*(1e-6^2))*0.001^2   # number of pores per mm2

    area_std = np.std(data['C.S. area'])
    perimeter_av = np.average(data['Perimeter'])
    perimeter_std = np.std(data['Perimeter'])
    area_perimeter_av = np.average(data['C.S. area']/data['Perimeter'])
    area_perimeter_std = np.std(data['C.S. area']/data['Perimeter'])
    diameter_av = np.average(data['Circle equivalent dia'])
    diameter_std = np.std(data['Circle equivalent dia'])
    #print 'area based on diameter_av: ' + str(((diameter_av/2)^2*pi).n())
    #print 'C.S. area: ' + str(area_av)

    resdict = {}
    list_props = ['porosity', 'area_av', 'density_mm', 'area_std', 'perimeter_av', 'perimeter_std', 'area_perimeter_av', 'area_perimeter_std', 'diameter_av', 'diameter_std']
    for dummy in list_props:
        resdict[dummy] = eval(dummy)
    return resdict

In [6]:
def gsv_bump(vdict_in, fname, delimiter1=','):
    # Standard values
    vdict = cdict.copy()
    vdict[T_a] = 300.     # Assumption
    vdict[P_a] = 101325
    vdict[k_dv] = 1e-3
    vdict[D_va] = eq_Dva.rhs().subs(vdict)
    vdict.update(vdict_in)  # updating all entries occurring in vdict_in
    example1 = stat_bump(fname, delimiter1 = delimiter1)
    
    try:
        vdict[F_p] = example1['porosity']/100.
        vdict[A_p] = example1['area_av']*(1e-6^2)
        vdict[n_p] = (F_p/A_p).subs(vdict)  # For regular pore distribution we can derive the pore density from porosity and pore area
        vdict[s_p] = eq_sp_np.rhs().subs(vdict)
        rp_keyence = example1['diameter_av']/2*1e-6  
        vdict[r_p] = eq_rp_Ap.rhs().subs(vdict).n()
        rperror = abs(rp_keyence - vdict[r_p])/rp_keyence
        if rperror > 0.05:
            print 'mismatch by ' + str(rperror*100)+ '%! rp_keyence: ' + str(rp_keyence) + '; r_p:' + str(vdict[r_p])
        
    except Exception, e:
        print e
        vdict[g_sw] = NaN
        return vdict

    gsp = eq_gsw_gswmol_iso.rhs().subs(g_swmol == 1/eq_rsp_rp.rhs()).subs(vdict).n()
    gsw = eq_gsw_gswmol_iso.rhs().subs(eq_gswmol.subs(eq_rend_rp, eq_rsp_rp, eq_rvs_B)).subs(vdict).n()
    gsw_r0 = eq_gsw_gswmol_iso.rhs().subs(eq_gswmol.subs(r_end==0, eq_rsp_rp, eq_rvs_B)).subs(vdict).n()
    #gsw_r0_S = eq_gsw_gswmol_iso.rhs().subs(eq_gswmol.subs(r_end==0, eq_rsp_rp, eq_rvs_S)).subs(vdict).n()

    # Now computing gsw assuming 50% bigger stomatal area
    vdict1 = vdict.copy()
    vdict1[A_p] = 1.5*vdict[A_p]
    vdict1[r_p] = eq_rp_Ap.rhs().subs(vdict1)
    gsp50 = eq_gsw_gswmol_iso.rhs().subs(g_swmol == 1/eq_rsp_rp.rhs()).subs(vdict1).n()
    gsw50 = eq_gsw_gswmol_iso.rhs().subs(eq_gswmol.subs(eq_rend_rp, eq_rsp_rp, eq_rvs_B)).subs(vdict1).n()
    gsw_r050 = eq_gsw_gswmol_iso.rhs().subs(eq_gswmol.subs(r_end==0, eq_rsp_rp, eq_rvs_B)).subs(vdict1).n()    
    
    #vdict[g_sw] = gsw
    
    resdict = {}
    list_props = ['fname', 'gsp', 'gsw', 'gsw_r0', 'gsp50', 'gsw50', 'gsw_r050']
    
    for dummy in list_props:
        resdict[dummy] = eval(dummy)
    for dummy in example1.keys():
        resdict[str(dummy)] = example1[dummy]
    resdict['vdict'] = vdict
    return resdict

In [7]:
# Test
vdict1 = {}
vdict1[T_l] = 300
dict_print(gsv_bump(vdict1, path_data + '35_4/new_analysis/35_2c_matte_10x_bump.csv', delimiter1=';'))

area_av         1278.07271087  
area_perim      1.02329878749  
area_perim      8.10321519641  
area_std        362.405791974  
density_mm      29.8887533355  
diameter_a      39.8975434783  
diameter_s      5.95639748961  
fname           data/perforated_foils/35_4/new_analysis/35_2c_matte_10x_bump.csv
gsp             0.0376151390555144
gsp50           0.0564227085832717
gsw             0.0172677642706621
gsw50           0.0233278616128606
gsw_r0          0.0243512590402737
gsw_r050        0.0343492611501317
perimeter_      31.8588941464  
perimeter_      155.294034783  
porosity        3.82           
vdict           {g: 9.81000000000000, k_dv: 0.00100000000000000, c_pa: 1010, r_p: 0.0000201698581832273, Pr: 0.710000000000000, d_p: 0.0000250000000000000, T_l: 300, A_p: 1.2780727108695653e-09, sigm: 5.67000000000000e-8, s_p: 0.00018291364303436687, lambda_E: 2.45000000000000e6, M_w: 0.0180000000000000, F_p: 0.038199999999999998, R_mol: 8.31447200000000, a_sh: 2, n_p: 29888753.33548885

<h1>Results</h1>

In [8]:
# Creating named array for storing data of bump analyses
pathname = path_data + '63_7/'
fname = '63_7_1a_bump_a'
result_dict = gsv_bump(vdict1, pathname + fname + '.csv')

# Creating named array for storing data
names = [str(name) for name in result_dict.keys()]
formats = ['f4' for i in srange(len(names))]
for i in srange(len(names)):
    if names[i] == 'fname':
        formats[i] = 'S200'
    if names[i] == 'vdict':
        formats[i] = 'object'
           
results_array_b = np.array([],dtype = zip(names, formats))

# Arrays for computations with d_p=0
results_array_b0 = np.array([],dtype = zip(names, formats))


In [9]:
def fun_append(results_array, result_dict, print_names = ['density_mm', 'area_av', 'diameter_av', 'radius_area_av', 'gsp', 'gsp50', 'gsw', 'gsw50', 'gsw_r0', 'gsw_r050']):
    '''
    Append content of results_dict to results_array in the order of names in results_array.
    results_array.dtype.names must be equal to result_dict.keys()
    Example:
        sage: fun_append(results_array, result_dict)
    '''
    if not list(results_array.dtype.names) == result_dict.keys():
        print sorted(results_array.dtype.names)
        print sorted(result_dict.keys())
        return 'dictionary does not have the same entries as array'
    dtype1 = results_array.dtype
    try: 
        line1 = np.array(tuple([result_dict[name1] for name1 in dtype1.names]), dtype = dtype1)
        outarray = np.append(results_array, line1)
    except Exception, e:
        print e
        print 'dtype: ' + str(dtype1)
        print 'result_dict: ' + str(result_dict)
    for name1 in print_names:
        try:    print name1 + ': ' + str(outarray[name1][-1])
        except: pass
    return outarray

In [10]:
# Test
pathname = path_data + '63_7/'
fname = '63_7_1a_bump_a'
vdict1 = cdict.copy()
result_dict = gsv_bump(vdict1, pathname + fname + '.csv')
print 'number of holes per mm2: ' + str(result_dict['vdict'][n_p]*0.001^2)
blah = fun_append(results_array_b, result_dict)

number of holes per mm2: 64.29324922677597
density_mm: 64.2933
area_av: 1153.93
diameter_av: 38.2712
gsp: 0.0730541
gsp50: 0.109581
gsw: 0.0350147
gsw50: 0.0476831
gsw_r0: 0.0492183
gsw_r050: 0.0702128


In [11]:
def fun_printminmax(results_array, varname, pathname, fnameconds = [], list_files = None, print1 = True):
    '''
    Finds all entries in pathname that contain all elements of
    fnameconds and returns min and max of varname. Prepend 'vdict_' to varname
    if it is stored in vdict.
    '''

    poslist = []
    for i in srange(len(results_array)):
        if list_files: 
            if any(s in results_array['fname'][i] for s in list_files):
                poslist.append(i)            
        else:
            fname = results_array['fname'][i]
            if fname[0:len(pathname)] == pathname:
                fnameconds1 = [True]
                for cond in fnameconds:
                    fnameconds1.append(cond in fname)
                    #print fname
                    #print fnameconds1
                if False not in fnameconds1:
                    #print 'true!'
                    poslist.append(i)

    if varname[0:6] == 'vdict_':
        listval = [results_array['vdict'][pos1][var(varname[6:])] for pos1 in poslist]
        minval = min(listval)
        maxval = max(listval)      
    else:   
        minval = min(results_array[varname][poslist])
        maxval = max(results_array[varname][poslist])
    if print1 == True:
        print varname + ': ' + str((minval, maxval))
    return (minval, maxval)

In [12]:
def fun_calc_all(pathname = 'data/', \
                 list_fnames_b = None, \
                 list_printnames = ['density_mm', 'area_av', 'diameter_av', 'radius_area_av', 'gsp', 'gsp50', 'gsw', 'gsw50','gsw_r0', 'gsw_r050'], \
                 append1 = True, delimiter1 = ",", vdict1 = False):
    '''
    Runs all calculations, appends to arrays and prints summaries. 
    Boundary conditions are taken from cdict, unless provided as vdict1.
    '''
    global results_array_b, results_array_b0
   
    out_results_array_b = results_array_b.copy()
    out_results_array_b0 = results_array_b0.copy()
    
    if list_fnames_b is None:  # finding all csv files with "bump" in their names
        list_files = os.listdir(pathname)
        list_fnames_b1 =[s for s in list_files if "csv" in s and "bump" in s]
        list_fnames_b = sorted(list_fnames_b1)
    print list_fnames_b

    for fname in list_fnames_b:
        if vdict1 is False:
            vdict2 = cdict.copy()
        else:
            vdict2 = vdict1.copy()
            
        if fname[-4:] != '.csv':
            fullpath = pathname + fname + '.csv'
        else:
            fullpath = pathname + fname
        result_dict = gsv_bump(vdict2, fullpath, delimiter1 = delimiter1)
        #print 'number of holes per mm2: ' + str(result_dict['vdict'][n_p]*0.001^2)
        if NaN not in result_dict.values():
            out_results_array_b = fun_append(out_results_array_b, result_dict, print_names = [])

        # Same for d_p = 0
        vdict2[d_p] = 1e-12
        result_dict = gsv_bump(vdict2, fullpath, delimiter1 = delimiter1)
        #print 'number of holes per mm2: ' + str(result_dict['vdict'][n_p]*0.001^2)
        if NaN not in result_dict.values():
            out_results_array_b0 = fun_append(out_results_array_b0, result_dict, print_names = [])    
            
    print 'number of holes per mm2: ' + str(result_dict['vdict'][n_p]*0.001^2)
    print 'min/max BUMP:'
    for varname in list_printnames:
        try: blah = fun_printminmax(out_results_array_b, varname, pathname, list_files = list_fnames_b);
        except: pass

    print ''
    print 'min/max BUMP at d_p = 0:'
    for varname in list_printnames[4:]:
        try: blah = fun_printminmax(out_results_array_b0, varname, pathname, list_files = list_fnames_b);
        except: pass

    if append1:
        results_array_b = out_results_array_b
        results_array_b0 = out_results_array_b0
        
pathname1 = path_data + '63_7/'
fun_calc_all(pathname = pathname1, append1 = False)

['63_7_1a_bump_a.csv', '63_7_1b_bump_a.csv', '63_7_1c_bump_a.csv', '63_7_2a_bump_a.csv', '63_7_2c_bump_a.csv']
number of holes per mm2: 64.9584048980769
min/max BUMP:
density_mm: (64.293251, 64.987022)
area_av: (1104.2205, 1240.043)
diameter_av: (37.327999, 39.672829)
gsp: (0.070661321, 0.079208426)
gsp50: (0.10599198, 0.11881264)
gsw: (0.034224734, 0.03735587)
gsw50: (0.046637036, 0.050835587)
gsw_r0: (0.047885068, 0.052923851)
gsw_r050: (0.068322606, 0.075518593)

min/max BUMP at d_p = 0:
gsp: (1766533.0, 1980210.6)
gsp50: (2649799.5, 2970316.0)
gsw: (0.06637188, 0.070698187)
gsw50: (0.083281219, 0.088852197)
gsw_r0: (0.1485592, 0.15948573)
gsw_r050: (0.19224234, 0.20724706)


<h2>7 holes per mm2</h2>

In [13]:
pathname1 = path_data + '7/'
fun_calc_all(pathname = pathname1, delimiter1 = ",")

['7_1a_bump_a.csv', '7_1b_bump_a.csv', '7_1c_bump_a.csv', '7_2a_bump_a.csv', '7_2b_bump_a.csv']
number of holes per mm2: 7.805322698787334
min/max BUMP:
density_mm: (7.7868133, 7.8053226)
area_av: (1040.2625, 1279.8959)
diameter_av: (36.163334, 40.327499)
gsp: (0.0079858322, 0.0098370481)
gsp50: (0.011978748, 0.014755572)
gsw: (0.0037912941, 0.0044257501)
gsw50: (0.0051085786, 0.005937987)
gsw_r0: (0.0052035498, 0.0061923591)
gsw_r050: (0.0072833379, 0.0086374423)

min/max BUMP at d_p = 0:
gsp: (199645.8, 245926.2)
gsp50: (299468.69, 368889.31)
gsw: (0.0072181099, 0.008045448)
gsw50: (0.0089072585, 0.0099367779)
gsw_r0: (0.014935465, 0.016713232)
gsw_r050: (0.018580968, 0.020831596)


In [14]:
pathname1 = path_data + '7/new_analysis/'
fun_calc_all(pathname = pathname1, delimiter1 = ";")

['7_3a_matte_bump.csv', '7_3a_shiny_bump.csv', '7_3b_matte_bump.csv', '7_3b_shiny_bump.csv', '7_3c_matte_bump.csv', '7_3c_shiny_bump.csv']
number of holes per mm2: 7.797169108044727
min/max BUMP:
density_mm: (7.15868, 7.8058972)
area_av: (889.82886, 1886.3206)
diameter_av: (33.438446, 48.809765)
gsp: (0.006272472, 0.014474936)
gsp50: (0.0094087087, 0.021712404)
gsw: (0.0030942585, 0.0058542057)
gsw50: (0.0041823331, 0.0077960044)
gsw_r0: (0.0041860864, 0.0085008573)
gsw_r050: (0.0058728051, 0.011785813)

min/max BUMP at d_p = 0:
gsp: (156811.8, 361873.41)
gsp50: (235217.7, 542810.06)
gsw: (0.006106779, 0.0098297065)
gsw50: (0.0075291856, 0.012163347)
gsw_r0: (0.012584971, 0.02059721)
gsw_r050: (0.015626986, 0.025779074)


<h2>35.4 holes/mm2</h2>

In [15]:
pathname1 = path_data + '35_4/'
fun_calc_all(pathname = pathname1, delimiter1 = ",")

['35_4_1a_bump_a.csv', '35_4_1b_bump_a.csv', '35_4_2a_bump_a.csv', '35_4_2b_bump_a.csv', '35_4_2c_bump_a.csv']
number of holes per mm2: 31.165016777105922
min/max BUMP:
density_mm: (29.87863, 38.201073)
area_av: (1025.1473, 1321.1671)
diameter_av: (35.967609, 40.90979)
gsp: (0.03016104, 0.04969728)
gsp50: (0.045241561, 0.07454592)
gsw: (0.014615324, 0.022749657)
gsw50: (0.019816091, 0.030777043)
gsw_r0: (0.020159025, 0.032265101)
gsw_r050: (0.028489189, 0.045647252)

min/max BUMP at d_p = 0:
gsp: (754026.0, 1242432.0)
gsp50: (1131039.0, 1863648.0)
gsw: (0.028355937, 0.041955315)
gsw50: (0.03526035, 0.052418593)
gsw_r0: (0.060789455, 0.091984354)
gsw_r050: (0.076938085, 0.11774994)


In [16]:
pathname1 = path_data + '35_4/new_analysis/'
fun_calc_all(pathname = pathname1, delimiter1 = ";")

['35_2a_matte_10x_bump.csv', '35_2a_matte_20x_bump.csv', '35_2a_shiny_10x_bump.csv', '35_2a_shiny_20x_bump.csv', '35_2b_matte_10x_bump.csv', '35_2b_matte_20x_bump.csv', '35_2b_shiny_10x_bump.csv', '35_2b_shiny_20x_bump.csv', '35_2c_matte_10x_bump.csv', '35_2c_matte_20x_bump.csv', '35_2c_shiny_10x_bump.csv', '35_2c_shiny_20x_bump.csv']
number of holes per mm2: 31.226058174159085
min/max BUMP:
density_mm: (27.281227, 33.849636)
area_av: (709.81348, 1572.725)
diameter_av: (29.252899, 44.566666)
gsp: (0.021810872, 0.048358101)
gsp50: (0.032716308, 0.072537154)
gsw: (0.011516015, 0.02108092)
gsw50: (0.015721614, 0.028402165)
gsw_r0: (0.015341002, 0.030394502)
gsw_r050: (0.021772785, 0.042844273)

min/max BUMP at d_p = 0:
gsp: (545271.81, 1208952.5)
gsp50: (817907.75, 1813428.8)
gsw: (0.024398038, 0.037474178)
gsw50: (0.030265512, 0.046771899)
gsw_r0: (0.051716745, 0.081822142)
gsw_r050: (0.065091029, 0.10466488)


<h2>63.7 holes per mm1</h2>

In [17]:
pathname1 = path_data + '63_7/'
fun_calc_all(pathname = pathname1, delimiter1 = ",")

['63_7_1a_bump_a.csv', '63_7_1b_bump_a.csv', '63_7_1c_bump_a.csv', '63_7_2a_bump_a.csv', '63_7_2c_bump_a.csv']
number of holes per mm2: 64.9584048980769
min/max BUMP:
density_mm: (64.293251, 64.987022)
area_av: (1104.2205, 1240.043)
diameter_av: (37.327999, 39.672829)
gsp: (0.070661321, 0.079208426)
gsp50: (0.10599198, 0.11881264)
gsw: (0.034224734, 0.03735587)
gsw50: (0.046637036, 0.050835587)
gsw_r0: (0.047885068, 0.052923851)
gsw_r050: (0.068322606, 0.075518593)

min/max BUMP at d_p = 0:
gsp: (1766533.0, 1980210.6)
gsp50: (2649799.5, 2970316.0)
gsw: (0.06637188, 0.070698187)
gsw50: (0.083281219, 0.088852197)
gsw_r0: (0.1485592, 0.15948573)
gsw_r050: (0.19224234, 0.20724706)


In [18]:
pathname1 = path_data + '63_7/new_analysis/'
fun_calc_all(pathname = pathname1, delimiter1 = ";")

['63_3a_matte_10x_bump.csv', '63_3a_matte_20x_bump.csv', '63_3a_shiny_10x_bump.csv', '63_3a_shiny_20x_bump.csv', '63_3b_matte_10x_bump.csv', '63_3b_matte_20x_bump.csv', '63_3b_shiny_10x_bump.csv', '63_3b_shiny_20x_bump.csv', '63_3c_matte_10x_bump.csv', '63_3c_matte_20x_bump.csv', '63_3c_shiny_10x_bump.csv', '63_3c_shiny_20x_bump.csv']
number of holes per mm2: 62.4827827733611
min/max BUMP:
density_mm: (52.061836, 68.828247)
area_av: (858.94696, 1150.6147)
diameter_av: (32.76091, 38.138435)
gsp: (0.046231173, 0.077386752)
gsp50: (0.069346756, 0.11608012)
gsw: (0.023610478, 0.037254516)
gsw50: (0.032237127, 0.05077536)
gsw_r0: (0.032135971, 0.052347913)
gsw_r050: (0.045775522, 0.074764237)

min/max BUMP at d_p = 0:
gsp: (1155779.2, 1934668.8)
gsp50: (1733669.0, 2902003.0)
gsw: (0.04825405, 0.071837664)
gsw50: (0.060241506, 0.090253912)
gsw_r0: (0.10540348, 0.16179004)
gsw_r050: (0.13467191, 0.21005583)


# Summary

In [19]:
results_array_b['fname']

array(['data/perforated_foils/7/7_1a_bump_a.csv',
       'data/perforated_foils/7/7_1b_bump_a.csv',
       'data/perforated_foils/7/7_1c_bump_a.csv',
       'data/perforated_foils/7/7_2a_bump_a.csv',
       'data/perforated_foils/7/7_2b_bump_a.csv',
       'data/perforated_foils/7/new_analysis/7_3a_matte_bump.csv',
       'data/perforated_foils/7/new_analysis/7_3a_shiny_bump.csv',
       'data/perforated_foils/7/new_analysis/7_3b_matte_bump.csv',
       'data/perforated_foils/7/new_analysis/7_3b_shiny_bump.csv',
       'data/perforated_foils/7/new_analysis/7_3c_matte_bump.csv',
       'data/perforated_foils/7/new_analysis/7_3c_shiny_bump.csv',
       'data/perforated_foils/35_4/35_4_1a_bump_a.csv',
       'data/perforated_foils/35_4/35_4_1b_bump_a.csv',
       'data/perforated_foils/35_4/35_4_2a_bump_a.csv',
       'data/perforated_foils/35_4/35_4_2b_bump_a.csv',
       'data/perforated_foils/35_4/35_4_2c_bump_a.csv',
       'data/perforated_foils/35_4/new_analysis/35_2a_matte_10x_bump

In [20]:
list_names = ['63_7', '35_4']
list_varnames1 = ['density_mm', 'area_av', 'diameter_av']
list_varnames2 = ['gsw_r0', 'gsw_r050']

results_array = results_array_b.copy()

for name1 in list_names:
    print ''
    print name1
    print 'orig. foils: '
    pathname = 'data/perforated_foils/' + name1 + '/' + name1
    for varname in list_varnames1:
        blah = fun_printminmax(results_array, varname, pathname)
    for varname in list_varnames2:
        blah = fun_printminmax(results_array, varname, pathname, print1 = False)
        print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)
        blah = fun_printminmax(results_array_b0, varname, pathname, print1 = False)
        print varname + '(d_p = 0 um) = ' + str(blah)
    
    print ''    
    print 'shiny 10x: '
    pathname = 'data/perforated_foils/' + name1 + '/new_analysis/'
    for varname in list_varnames1:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['10x', 'shiny'])
    for varname in list_varnames2:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['10x', 'shiny'], print1 = False)
        print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)

    print ''    
    print 'matte 10x: '
    pathname = 'data/perforated_foils/' + name1 + '/new_analysis/'
    for varname in list_varnames1:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['10x', 'matte'])
    for varname in list_varnames2:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['10x', 'matte'], print1 = False)
        print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)

    print ''    
    print 'shiny 20x: '
    pathname = 'data/perforated_foils/' + name1 + '/new_analysis/'
    for varname in list_varnames1:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['20x', 'shiny'])
    for varname in list_varnames2:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['20x', 'shiny'], print1 = False)
        print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)
    
    print ''
    print 'matte 20x: '
    pathname = 'data/perforated_foils/' + name1 + '/new_analysis/'
    for varname in list_varnames1:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['20x', 'matte'])
    for varname in list_varnames2:
        blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['20x', 'matte'], print1 = False)
        print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)
        
name1 = '7'
print ''
print name1
print 'orig. foils: '
pathname = 'data/perforated_foils/' + name1 + '/' + name1
for varname in list_varnames1:
    blah = fun_printminmax(results_array, varname, pathname)
for varname in list_varnames2:
    blah = fun_printminmax(results_array, varname, pathname, print1 = False)
    print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)
    blah = fun_printminmax(results_array_b0, varname, pathname, print1 = False)
    print varname + '(d_p = 0 um) = ' + str(blah)

print ''    
print 'shiny: '
pathname = 'data/perforated_foils/' + name1 + '/new_analysis/'
for varname in list_varnames1:
    blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['shiny'])
for varname in list_varnames2:
    blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['shiny'], print1 = False)
    print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)

print ''    
print 'matte: '
pathname = 'data/perforated_foils/' + name1 + '/new_analysis/'
for varname in list_varnames1:
    blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['matte'])
for varname in list_varnames2:
    blah = fun_printminmax(results_array, varname, pathname, fnameconds = ['matte'], print1 = False)
    print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)


63_7
orig. foils: 
density_mm: (64.293251, 64.987022)
area_av: (1104.2205, 1240.043)
diameter_av: (37.327999, 39.672829)
gsw_r0(d_p = 25 um) = (0.047885068, 0.052923851)
gsw_r0(d_p = 0 um) = (0.1485592, 0.15948573)
gsw_r050(d_p = 25 um) = (0.068322606, 0.075518593)
gsw_r050(d_p = 0 um) = (0.19224234, 0.20724706)

shiny 10x: 
density_mm: (63.709175, 68.828247)
area_av: (1120.4038, 1150.6147)
diameter_av: (37.644753, 38.138435)
gsw_r0(d_p = 25 um) = (0.047509938, 0.052347913)
gsw_r050(d_p = 25 um) = (0.067765139, 0.074764237)

matte 10x: 
density_mm: (54.543026, 64.330421)
area_av: (925.5542, 1147.6686)
diameter_av: (34.081692, 38.124378)
gsw_r0(d_p = 25 um) = (0.034235783, 0.049013514)
gsw_r050(d_p = 25 um) = (0.048746478, 0.069921151)

shiny 20x: 
density_mm: (62.475056, 62.503353)
area_av: (1083.3384, 1108.9226)
diameter_av: (37.004349, 37.42487)
gsw_r0(d_p = 25 um) = (0.045208666, 0.046136644)
gsw_r050(d_p = 25 um) = (0.064463899, 0.065785736)

matte 20x: 
density_mm: (52.061836, 57

## Summary for Table 1 in Schymanski et al. (2016, HESSD)

In [21]:
list_names = ['63_7', '35_4', '7']
list_varnames1 = ['density_mm', 'area_av', 'diameter_av']
list_varnames2 = ['gsw_r0', 'gsw_r050']

array_summary = []
for name1 in list_names:
    print ''
    print name1
    pathname = 'data/perforated_foils/' + name1 + '/'
    for varname in list_varnames1:
        blah = fun_printminmax(results_array_b, varname, pathname)
    for varname in list_varnames2:
        blah = fun_printminmax(results_array_b, varname, pathname, print1 = False)
        print varname + '(d_p = ' + str(int(cdict[d_p]*1e6)) + ' um) = ' + str(blah)
        #blah = fun_printminmax(results_array_b0, varname, pathname, print1 = False)
        #print varname + '(d_p = 0 um) = ' + str(blah)


63_7
density_mm: (52.061836, 68.828247)
area_av: (858.94696, 1240.043)
diameter_av: (32.76091, 39.672829)
gsw_r0(d_p = 25 um) = (0.032135971, 0.052923851)
gsw_r050(d_p = 25 um) = (0.045775522, 0.075518593)

35_4
density_mm: (27.281227, 38.201073)
area_av: (709.81348, 1572.725)
diameter_av: (29.252899, 44.566666)
gsw_r0(d_p = 25 um) = (0.015341002, 0.032265101)
gsw_r050(d_p = 25 um) = (0.021772785, 0.045647252)

7
density_mm: (7.15868, 7.8058972)
area_av: (889.82886, 1886.3206)
diameter_av: (33.438446, 48.809765)
gsw_r0(d_p = 25 um) = (0.0041860864, 0.0085008573)
gsw_r050(d_p = 25 um) = (0.0058728051, 0.011785813)
